In [107]:
import sklearn
import numpy as np
import PIL
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
import time

# Function define

In [108]:
def img_comp(graphA, graphB):
    return np.sum(np.sqrt(np.sum(np.power(graphA - graphB, 2), axis=1)))

def get_img_folder(path):
    img_list = []
    file_list = []
    for filename in os.listdir(path):
        if not filename.endswith('.jpg') and not filename.endswith('.jpeg') and not filename.endswith('.png'):
            continue
        image = cv2.resize(cv2.imread(os.path.join(path, filename), 0), (256, 256), interpolation=cv2.INTER_AREA)
        # area = image.shape[0] * image.shape[1]
        img = cv2.calcHist([image], [0], None, [256], [0.0, 255.0]) / 255
        # img = cv2.calcHist([image], [0, 1, 2], None, [256, 256, 256], [0.0, 255.0, 0.0, 255.0, 0.0, 255.0]) / area
        img_list.append(img.flatten())
        file_list.append(filename)
    return img_list, file_list

# Trainset import

In [109]:
man_folder_path = './train/manmade_training'
man_img_list, _ = get_img_folder(man_folder_path)
nat_folder_path = './train/natural_training'
nat_img_list, _ = get_img_folder(nat_folder_path)
trainset = pd.DataFrame({"image":man_img_list+nat_img_list, "type":[0]*len(man_img_list)+[1]*len(nat_img_list)})

# Testset import

In [110]:
man_folder_path = './test/manmade_test'
man_img_list, man_file = get_img_folder(man_folder_path)
nat_folder_path = './test/natural_test'
nat_img_list, nat_file = get_img_folder(nat_folder_path)
testset = pd.DataFrame({"image":man_img_list+nat_img_list, "type":[0]*len(man_img_list)+[1]*len(nat_img_list), "filename":man_file+nat_file, "prediction":0, "confidence":0})

In [111]:
del man_folder_path, man_file, nat_file, nat_folder_path, man_img_list, nat_img_list, _

In [114]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=11).fit(list(trainset["image"]), list(trainset["type"]))
predictions_labels = clf.predict(testset["image"].values.tolist())

In [ ]:
# predictions = [0] * len(testset)
# confidence = [0] * len(testset)
# for i in testset.index:
#     image = testset["image"][i]
#     distances = [0] * len(trainset)
#     for j in trainset.index:
#         distances[j] = img_comp(image, trainset["image"][j])
#     trainset["distance"] = distances
#     trainset = trainset.sort_values(by="distance")
#     res = trainset[:10]
#     man_cnt = res.loc[res["type"]==0].shape[0]
#     nat_cnt = 10 - man_cnt
#     predictions[i] = 0 if man_cnt > nat_cnt else 1
#     confidence[i] = man_cnt / 10 if man_cnt > nat_cnt else nat_cnt / 10
# testset["prediction"] = predictions
# testset["confidence"] = confidence
    

In [121]:
# predictions_labels
testset["prediction"] = predictions_labels
testset.loc[testset["type"]==testset["prediction"]].shape[0] / testset.shape[0]